# PneumoniaMNIST Medical Imaging Challenge

This notebook covers the following three tasks using the **PneumoniaMNIST** dataset from MedMNIST v2:
1. **CNN Classification with Comprehensive Analysis**: Training a ResNet-18 model and evaluating it with Accuracy, AUC, Confusion Matrix, and Grad-CAM.
2. **Medical Report Generation using Visual Language Models**: An Encoder-Decoder architecture to generate synthetic medical reports from X-ray images.
3. **Semantic Image Retrieval System**: A feature-based retrieval system using deep embeddings to find similar cases.

## Setup and Dependencies

In [ ]:
!pip install medmnist torch torchvision matplotlib scikit-learn captum

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.models as models
import medmnist
from medmnist import INFO, Evaluator
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity
from captum.attr import LayerGradCam

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Data Preparation

In [ ]:
data_flag = 'pneumoniamnist'
info = INFO[data_flag]
DataClass = getattr(medmnist, info['python_class'])

data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

train_dataset = DataClass(split='train', transform=data_transform, download=True)
val_dataset = DataClass(split='val', transform=data_transform, download=True)
test_dataset = DataClass(split='test', transform=data_transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=128, shuffle=False)

## Task 1: CNN Classification with Comprehensive Analysis
We use a ResNet-18 architecture adapted for 28x28 grayscale images.

In [ ]:
def get_resnet18(num_classes=1):
    model = models.resnet18(weights=None)
    model.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

model = get_resnet18().to(device)
pos_weight = torch.tensor([1214 / 3494]).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 5
for epoch in range(1, epochs + 1):
    model.train()
    train_loss = 0
    for data, target in train_loader:
        data, target = data.to(device).float(), target.to(device).float()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    model.eval()
    all_targets = []
    all_outputs = []
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device).float(), target.to(device).float()
            output = model(data)
            all_targets.append(target.cpu().numpy())
            all_outputs.append(torch.sigmoid(output).cpu().numpy())
    
    all_targets = np.concatenate(all_targets)
    all_outputs = np.concatenate(all_outputs)
    val_auc = roc_auc_score(all_targets, all_outputs)
    print(f"Epoch {epoch}: Train Loss: {train_loss/len(train_loader):.4f}, Val AUC: {val_auc:.4f}")

### Evaluation and Visualization

In [ ]:
model.eval()
test_targets = []
test_outputs = []
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device).float(), target.to(device).float()
        output = model(data)
        test_targets.append(target.cpu().numpy())
        test_outputs.append(torch.sigmoid(output).cpu().numpy())

test_targets = np.concatenate(test_targets)
test_outputs = np.concatenate(test_outputs)
test_auc = roc_auc_score(test_targets, test_outputs)
test_acc = np.mean((test_outputs > 0.5) == test_targets)
print(f"Test AUC: {test_auc:.4f}, Test Accuracy: {test_acc:.4f}")

cm = confusion_matrix(test_targets, test_outputs > 0.5)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=info['label'].values())
disp.plot()
plt.title("Confusion Matrix")
plt.show()

### Interpretability with Grad-CAM

In [ ]:
layer_gc = LayerGradCam(model, model.layer4)
samples_idx = [0, 1, 2, 3, 4]
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
for i, idx in enumerate(samples_idx):
    img_tensor, target = test_dataset[idx]
    img_tensor = img_tensor.to(device).unsqueeze(0).float()
    img_tensor.requires_grad = True
    
    attr = layer_gc.attribute(img_tensor, target=0)
    upsampled_attr = LayerGradCam.interpolate(attr, (28, 28))
    
    img_np = img_tensor.squeeze().detach().cpu().numpy()
    heatmap = upsampled_attr.squeeze().detach().cpu().numpy()
    
    axes[i].imshow(img_np, cmap='gray')
    axes[i].imshow(heatmap, cmap='jet', alpha=0.4)
    axes[i].set_title(f"L:{target[0]} (P:{test_outputs[idx][0]:.2f})")
    axes[i].axis('off')
plt.suptitle("Grad-CAM Interpretability")
plt.show()

## Task 2: Medical Report Generation
We implement a small Encoder-Decoder (CNN + GRU) to generate reports based on image features.

In [ ]:
class ReportDataset(Dataset):
    def __init__(self, split='train', transform=None):
        self.dataset = DataClass(split=split, transform=transform, download=True)
        self.reports_normal = [
            "The lungs are clear. No evidence of pneumonia.",
            "Normal chest X-ray. No consolidations found.",
            "The heart size is normal. Lungs are clear."
        ]
        self.reports_pneumonia = [
            "Opacities observed in the lungs, consistent with pneumonia.",
            "Bilateral consolidations found. Findings suggest pneumonia.",
            "Patchy opacities in lung fields. Suggests pneumonia."
        ]
        self.vocab = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        all_text = " ".join(self.reports_normal + self.reports_pneumonia)
        words = sorted(list(set(all_text.replace(".", "").lower().split())))
        for i, word in enumerate(words): self.vocab[word] = i + 4
        self.inv_vocab = {v: k for k, v in self.vocab.items()}
        
    def tokenize(self, text):
        tokens = [self.vocab["<SOS>"]] + [self.vocab.get(w, self.vocab["<UNK>"]) for w in text.replace(".", "").lower().split()] + [self.vocab["<EOS>"]]
        return torch.tensor(tokens)

    def __len__(self): return len(self.dataset)
    def __getitem__(self, idx):
        img, target = self.dataset[idx]
        report = random.choice(self.reports_normal if target[0] == 0 else self.reports_pneumonia)
        return img, self.tokenize(report)

def collate_fn(batch):
    imgs, reports = zip(*batch)
    return torch.stack(imgs), nn.utils.rnn.pad_sequence(reports, batch_first=True, padding_value=0)

class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.gru = nn.GRU(embed_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, features, reports):
        embeddings = self.embed(reports[:, :-1])
        inputs = torch.cat((features.view(features.size(0), 1, -1), embeddings), 1)
        hiddens, _ = self.gru(inputs)
        return self.linear(hiddens)

    def sample(self, features, max_len=15):
        sampled_ids = []
        inputs = features.view(features.size(0), 1, -1)
        states = None
        for _ in range(max_len):
            hiddens, states = self.gru(inputs, states)
            outputs = self.linear(hiddens.squeeze(1))
            _, predicted = outputs.max(1)
            sampled_ids.append(predicted.item())
            inputs = self.embed(predicted).unsqueeze(1)
            if predicted.item() == 2: break
        return sampled_ids

rpt_dataset = ReportDataset(split='train', transform=data_transform)
rpt_loader = DataLoader(rpt_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
encoder = nn.Sequential(*(list(model.children())[:-1]))
decoder = DecoderRNN(512, 512, len(rpt_dataset.vocab)).to(device)
opt_dec = optim.Adam(decoder.parameters(), lr=0.001)
crit_dec = nn.CrossEntropyLoss(ignore_index=0)

for epoch in range(1, 4):
    for imgs, reports in rpt_loader:
        imgs, reports = imgs.to(device).float(), reports.to(device)
        with torch.no_grad(): feats = encoder(imgs)
        outputs = decoder(feats, reports)
        loss = crit_dec(outputs.view(-1, len(rpt_dataset.vocab)), reports.view(-1))
        opt_dec.zero_grad(); loss.backward(); opt_dec.step()
    print(f"Decoder Epoch {epoch} complete.")

In [ ]:
print("Generated Reports for Test Samples:")
test_rpt_dataset = ReportDataset(split='test', transform=data_transform)
for i in range(3):
    img, _ = test_rpt_dataset[i]
    feat = encoder(img.to(device).unsqueeze(0).float())
    ids = decoder.sample(feat)
    report = [test_rpt_dataset.inv_vocab[idx] for idx in ids if idx not in [1, 2]]
    print(f"Sample {i} (Label {test_rpt_dataset.dataset[i][1][0]}): {' '.join(report)}")

## Task 3: Semantic Image Retrieval System

In [ ]:
def extract_features(model, loader):
    model.eval()
    feats, labs = [], []
    with torch.no_grad():
        for imgs, targets in loader:
            f = model(imgs.to(device).float()).view(imgs.size(0), -1)
            feats.append(f.cpu().numpy()); labs.append(targets.numpy())
    return np.concatenate(feats), np.concatenate(labs)

train_feats, train_labs = extract_features(encoder, train_loader)
test_feats, test_labs = extract_features(encoder, test_loader)

query_idx = 0
sims = cosine_similarity(test_feats[query_idx].reshape(1, -1), train_feats).flatten()
top_k = sims.argsort()[-5:][::-1]

fig, axes = plt.subplots(1, 6, figsize=(18, 3))
axes[0].imshow(test_dataset[query_idx][0][0], cmap='gray')
axes[0].set_title("Query")
for i, idx in enumerate(top_k):
    axes[i+1].imshow(train_dataset[idx][0][0], cmap='gray')
    axes[i+1].set_title(f"Sim: {sims[idx]:.2f}")
    axes[i+1].axis('off')
plt.show()